# Traveling Salesman Problem Brute Force
## for mapping problems involving the same starting and ending coordinate

Given a set of `(long, lat)` coordinates, the Python program will brute force the shortest path from the first coordinate, across all coordinates, returning to the first coordinate.

In [55]:
###
long_lat = [(-80.3055455567875, 35.075461583165),
            (-81.5783244282868, 35.0675097680372),
            (-81.2285531353557, 35.3374363367321),
            (-80.5678740264858, 35.0038673621631),
            (-79.9654901331045, 34.7408191786722),
            (-79.3048110242347, 35.0516038135768)]
###

In [56]:
# imports
from math import radians, cos, sin, asin, sqrt
from itertools import permutations
from pprint import pp

In [57]:
# https://stackoverflow.com/a/4913653
# haversine distance function between two (long, lat) coordinates
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers is 6371
    km = 6371 * c
    return km

Generate a complete graph, $K_n$, where $n$ is the number of coordinates. We'll use the graph to compute all distances between coordinates so we can quickly compute each path length later.

The total number of edges in a complete graph is given by this formula:

$$\frac{n\cdot(n-1)}{2} = \left(\frac{1}{2}\right)\cdot\left(n^{2}-n\right)$$

Therefore, this computation grows exponentially with $n$.

In [58]:
def generate_complete_graph(n):
    edges = []
    for i in range(0, n):
        for j in range(i + 1, n):
            edges.append((i, j))
    return edges

pp(generate_complete_graph(4))

[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)]


Compute all edge distances over the complete graph using the Haversine function and store in a list of tuples. Each tuple is `(node_0, node_1, distance)`.

In [59]:
distances = [(edge[0], edge[1],
              haversine(long_lat[edge[0]][0], long_lat[edge[0]][1],
                        long_lat[edge[1]][0], long_lat[edge[1]][1]))
             for edge in generate_complete_graph(len(long_lat))]

pp(distances)

[(0, 1, 115.83298297185026),
 (0, 2, 88.77493861027976),
 (0, 3, 25.174613358411023),
 (0, 4, 48.43724128963063),
 (0, 5, 91.1198695601412),
 (1, 2, 43.71310888129238),
 (1, 3, 92.26864869788629),
 (1, 4, 151.495398718817),
 (1, 5, 206.93646304711461),
 (2, 3, 70.58346629665652),
 (2, 4, 132.75440602622768),
 (2, 5, 177.6703028552106),
 (3, 4, 62.25313661345294),
 (3, 5, 115.12943469379357),
 (4, 5, 69.4608439474778)]


When generating our paths, `(2, 3)` and `(3, 2)` refer to the same edge. These helper functions ensure that we look them up properly in our list of distances.

In [60]:
def greater_or(p):
    if p[0] > p[1]:
        return (p[1], p[0])
    return p

dist_dict = {}
for d in distances:
    if d[0] not in dist_dict:
        dist_dict[d[0]] = []
    dist_dict[d[0]].append((d[1], d[2]))

def lookup_distance(p):
    # p = (i1, i2)
    p = greater_or(p)
    start_node = p[0]
    end_node_i = p[1] - p[0] - 1
    dist = dist_dict[start_node][end_node_i][1]
    return dist

The number of paths in a graph totals to $n!$ where $n$ is again the number of nodes in the graph. So, for a graph of 6 nodes, there are $6! = 720$ paths.

All paths can be generated by permutating all nodes. For example, if `[1, 2, 3]` is the list of nodes in our graph, then all paths would be:

```
[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]
```

...where the total number of paths here happens to be $3! = 6$ paths.

For this problem specifically, however, we are fixing the first and last node of our path since we start and end up at the same node. Therefore, the number of paths is $(n-1)!$:

```
[1, 2, 3, 1]
[1, 3, 2, 1]
```

...where the total number of paths is $(3 - 1)! = 2! = 2$.

In [61]:
def generate_paths(n):
    return list(permutations(range(1, n + 1)))

pp(generate_paths(3))

# list of lat_long path permutations        
midpaths = generate_paths(len(long_lat) - 1) # we're fixing one node

# begin and end all paths with the starting node
paths = []
for path in midpaths:
    paths.append([0] + list(path) + [0])

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]


Lastly, we just need to actually compute the distances across all paths to find the smallest one. We find the distance of a path by jumping from one node to the next and adding up the distance as we go along. Our helper functions ensure that even though we're permutating all possible paths, edges like `(2, 3)` and `(3, 2)` will refer to the same edge.

Once we have all our path distances, we sort by the smallest, and voila!

In [62]:
distance_per_path = []
for path in paths:
    path_dist = 0
    for i in range(0, 5):
        path_dist += lookup_distance((path[i], path[i + 1]))
    distance_per_path.append((path, path_dist))

sorted_distance_per_path = sorted(distance_per_path, key=lambda tup: tup[1])

print(sorted_distance_per_path[0])

([0, 5, 4, 3, 2, 1, 0], 337.1304252990208)
